<a href="https://colab.research.google.com/github/nirb28/ee-predict/blob/main/cd_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [206]:
def resolve_path_gdrive(relativePath):
    if os.path.exists('/content/drive'):
        return '/content/drive/MyDrive/work/gdrive-workspaces/git/ee-predict/' + relativePath
    else:
        return relativePath
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [207]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np, os, csv
import warnings
warnings.filterwarnings('ignore')

In [208]:
## All of these are tunable paramaters
MODEL_REFRESH_STRATEGY = "RETRAIN" # Retrain model after every loop: RETRAIN, PRETRAIN
OPTIMIZE_STRATEGY = "NEWLY_ADDED" # Only optimize the newly added catalyst: NEWLY_ADDED, ALL
MODEL_FIT_USE_COLS = "models/high_corr_cols.txt" # Use all the columns: "ALL", "models/high_corr_cols.txt"
DEFAULT_PCA_DIMENSION = 4
START_SAMPLE_SIZE = 3
DATAFILE_NAME = "merged_large_catalyst.csv" # "reduced_dim_space_ddG.csv", "merged_large_catalyst.csv"
CD_STEP_SIZE = 0.001
CD_ITERATIONS_PER_STEP = 2
CD_ITERATIONS = 10
## *************************************************

In [209]:
df = pd.read_csv(resolve_path_gdrive('data/' + DATAFILE_NAME))
## removing ddG = 0 and indexing catalysts
df = df[df['ddG'] != 0]
df.set_index('Catalyst', inplace=True)

if DATAFILE_NAME == "reduced_dim_space_ddG.csv":
  X_COLS = ['x', 'y', 'z']
else:
  if MODEL_FIT_USE_COLS == "ALL":
    X_COLS = df.columns[~df.columns.isin(['Catalyst', 'ddG'])]
  else:
    with open(resolve_path_gdrive(MODEL_FIT_USE_COLS)) as f:
      reader = csv.reader(f)
      X_COLS = list(reader)[0][:-1]
df.head()

,1,2,3,4,5,6,7,8,9,10,...,3966,3967,3968,3969,3970,3971,3972,3973,3974,ddG
Catalyst,,,,,,,,,,,,,,,,,,,,,
1_1_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633,-0.672194,1.226289
1_1_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.602624,-0.471616,-0.798242,-0.277709,-0.280445,-0.279338,-0.246094,-1.153164,-1.146684,0.719002
1_11_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633,-0.672194,0.631877
1_2_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633,-0.672194,0.216792
1_2_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.602624,-0.471616,-0.798242,-0.277709,-0.280445,-0.279338,-0.246094,-1.153164,-1.146684,0.111677


In [210]:
# method to make a regression model from the provided catalysts
from sklearn.cross_decomposition import PLSRegression
def make_pls_model(df_catalysts, PCA_DIMENSION):
    X = df_catalysts[X_COLS]
    y = df_catalysts['ddG']
    model = PLSRegression(n_components=PCA_DIMENSION)
    print("PLS: Fitting model with {} rows and {} columns. PCA dimensions are {}"
        .format(len(X), len(X.columns), PCA_DIMENSION))
    # fitting the model
    return model.fit(X, y)

## Importing model created in "eda.ipynb"
from joblib import dump, load
def load_model(path):
  return load(path)

pls_large_saved_model = load_model(resolve_path_gdrive('models/pls_large.joblib'))
print(pls_large_saved_model)
## Read the dimension directly from the model so that if the model changes
## the dimension will be reflected automatically
PCA_DIMENSION = pls_large_saved_model.n_components

from sklearn.linear_model import LinearRegression
def make_lr_model(df_catalysts):
    X = df_catalysts[X_COLS]
    y = df_catalysts['ddG']
    model = LinearRegression()
    print("LR: Fitting model with {} rows and {} columns."
        .format(len(X), len(X.columns)))
    # fitting the model
    return model.fit(X, y)

def make_model(df_catalysts, PCA_DIMENSION=DEFAULT_PCA_DIMENSION):
    return make_pls_model(df_catalysts, PCA_DIMENSION)
    #return make_lr_model(df_catalysts)

def predict_ee(model, properties):
    return model.predict(properties.reshape(1,-1))

## Add a predicted ddG column to the df
#df['pred_ddG'] = df.apply(lambda x: predict_ee(pls_large_saved_model, np.array(x[X_COLS]))[0][0], axis=1)

PLSRegression(n_components=4)


In [211]:
top5_ddg = df.sort_values('ddG', ascending=False).head(5)
top5_ddG_values = top5_ddg['ddG']

top5_ddG_values

Catalyst
11_1_4    1.640250
21_1_4    1.634937
18_1_1    1.629666
23_1_4    1.588915
21_1_1    1.564636
Name: ddG, dtype: float64

In [212]:
# start with a few randomly selected catalyst
random_catalyst_df = df.sample(START_SAMPLE_SIZE)
## This will be used to track progress of coordinate descent
print("The max value (target) of the ddG is at {}. Currently we have reached {}.".format(df['ddG'].max(), random_catalyst_df['ddG'].max()))
random_catalyst_df

The max value (target) of the ddG is at 1.640249758. Currently we have reached 0.289947175.


,1,2,3,4,5,6,7,8,9,10,...,3966,3967,3968,3969,3970,3971,3972,3973,3974,ddG
Catalyst,,,,,,,,,,,,,,,,,,,,,
7_3_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.602624,-0.471616,-0.798242,-0.277709,-0.280445,-0.279338,-0.246094,-1.153164,-1.146684,0.036733
21_6_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633,-0.672194,0.289947
3_2_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633,-0.672194,0.033176


In [213]:
original_ee = df['ddG']
# Function to optimize catalyst properties using coordinate descent
def optimize_catalysts(catalysts, pls_model=None, iterations=CD_ITERATIONS,
                       cd_iterations=CD_ITERATIONS_PER_STEP, step_size=CD_STEP_SIZE):
    if pls_model == None:
        pls_model = make_model(random_catalyst_df)
    optimized_catalysts = np.copy(catalysts)
    for _ in range(iterations):
        for i in range(len(optimized_catalysts)):
            ## we have few options for the original_ee. We can use from the dataset or get the prediction
            ## from the model. The prediction makes sense as we want to traverse in the direction where
            ## ee improves according to the model

            ## when using ddG from dataset
            #original_ee = df.iloc[df.index.get_loc(catalysts[i:i+1].index[0])]['ddG']
            ## when using the model
            original_ee = predict_ee(pls_model, optimized_catalysts[i])
            for x in range(len(optimized_catalysts[i])):
                for cd in range(cd_iterations):
                    old_value = optimized_catalysts[i, x]
                    optimized_catalysts[i, x] = old_value + step_size
                    new_ee = predict_ee(pls_model, optimized_catalysts[i])
                    if new_ee < original_ee:
                        optimized_catalysts[i, x] = old_value - step_size
                        new_ee = predict_ee(pls_model, optimized_catalysts[i])
                    if new_ee < original_ee:
                        optimized_catalysts[i, x] = old_value
                        break
                    #print('Found a direction for higher ddG')
    return optimized_catalysts


In [214]:
if MODEL_REFRESH_STRATEGY == "RETRAIN":
  new_optimized_catalysts = optimize_catalysts(random_catalyst_df[X_COLS])
else:
  new_optimized_catalysts = optimize_catalysts(random_catalyst_df[X_COLS], pls_large_saved_model)

PLS: Fitting model with 3 rows and 62 columns. PCA dimensions are 4


In [215]:
# Perform k-nearest neighbors analysis
def recomputeKNN(df, removePointsDF = None):
  if len(removePointsDF) > 0:
    df.drop(removePointsDF.index, axis=0, inplace=True)
  num_neighbors = 1  # Number of neighbors to consider
  knn = NearestNeighbors(n_neighbors=num_neighbors)
  print('KNN: Fitting model with {} points.'.format(len(df)))
  knn.fit(df)  # Using all the catalysts
  return knn

knn = recomputeKNN(df[X_COLS], random_catalyst_df)

KNN: Fitting model with 316 points.


In [216]:
## Recursive Loop
## The method will get unique neighbors that are not already in the list.
## If the 1st neighbor is already in the list then we try the 2nd, 3rd and so on
## till a unique neighbor is found
def find_unique_neighbors_add(knn, point, addToDf, neighbor_number=1):
    distances, indices = knn.kneighbors([point], neighbor_number)
    neighbor_catalyst = df.iloc[[indices[0][neighbor_number-1]]]
    if neighbor_catalyst.index[0] in addToDf.index:
        #get next neighbor
        neighbor_catalyst = find_unique_neighbors_add(knn, point, addToDf, neighbor_number+1)
    return neighbor_catalyst

def process_neighbors(knn, for_points, addToDf):
    for i in range(len(for_points)):
        neighbor_catalyst = find_unique_neighbors_add(knn, for_points[i], addToDf)
        ## Add found catalysts into our batch (random_catalyst_df)
        if len(neighbor_catalyst) > 0:
            addToDf = pd.concat([addToDf, neighbor_catalyst], ignore_index=False, sort=True)
    return addToDf

random_catalyst_df = process_neighbors(knn, new_optimized_catalysts, random_catalyst_df)
random_catalyst_df[X_COLS]

,906,907,908,922,923,937,950,951,952,1113,...,3173,3524,3525,3547,3548,3628,3688,3916,3940,3951
Catalyst,,,,,,,,,,,,,,,,,,,,,
7_3_2,0.333333,0.333333,0.266667,0.333333,0.266667,0.400000,0.333333,0.333333,0.200000,0.333333,...,0.0500,-0.055,-0.0738,-0.04,-0.04,-0.09,-0.015,-0.0192,0.02,0.0
21_6_1,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.200000,0.200000,0.000000,...,-0.0288,0.000,0.0000,0.00,0.00,0.00,0.000,0.0000,0.00,0.0
3_2_1,0.000000,0.053333,0.120000,0.173333,0.120000,0.160000,0.040000,0.146667,0.186667,0.200000,...,0.0000,0.000,0.0000,0.00,0.00,0.00,0.000,0.0000,0.00,0.0
7_2_2,0.000000,0.041379,0.103448,0.124138,0.089655,0.117241,0.020690,0.179310,0.234483,0.193103,...,0.0000,0.000,0.0000,0.00,0.00,0.00,0.000,0.0000,0.00,0.0
21_6_2,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.200000,0.200000,0.000000,...,-0.0288,0.000,0.0000,0.00,0.00,0.00,0.000,0.0000,0.00,0.0
3_11_1,0.000000,0.166667,0.333333,0.416667,0.333333,0.333333,0.000000,0.000000,0.000000,0.333333,...,0.0000,0.000,-0.0125,0.00,0.00,0.00,0.000,0.0875,0.00,0.0


In [192]:
# We will now do all this in a single loop

## Maximum of entire dataset
max_ddG = df['ddG'].max()
print("The max value (target) of the ddG is {}. Model training strategy is {}".format(max_ddG, MODEL_REFRESH_STRATEGY))

## Get random rows
#random_catalyst_df = df.loc[not_highest_index].sample(START_SAMPLE_SIZE)
## Get lowest ddG to start
#random_catalyst_df = df.sort_values('ddG').head(START_SAMPLE_SIZE)
## Static list to start



while(len(random_catalyst_df) < len(df)):
        print("The current maximum ddg is at: " + str(random_catalyst_df['ddG'].max()))
        print(str(len(random_catalyst_df)) + " out of " + str(len(df)))
        what_to_optimize = random_catalyst_df[X_COLS]
        if OPTIMIZE_STRATEGY == "NEWLY_ADDED":
            what_to_optimize = what_to_optimize.tail(START_SAMPLE_SIZE)
        new_optimized_catalysts = optimize_catalysts(what_to_optimize,
            pls_large_saved_model if MODEL_REFRESH_STRATEGY == "PRETRAIN" else None)
        random_catalyst_df = process_neighbors(knn, new_optimized_catalysts, random_catalyst_df)
        # recompute knn so that we dont find the same neighbors in the next loop around
        knn = recomputeKNN(df[X_COLS], random_catalyst_df)
        if random_catalyst_df['ddG'].max() >= max_ddG:
            print("Found the highest ddG {}. Exiting.".format(str(random_catalyst_df['ddG'].max())))
            break



The max value (target) of the ddG is 1.640249758. Model training strategy is RETRAIN
The current maximum ddg is at: 0.349790382
6 out of 319
PLS: Fitting model with 6 rows and 62 columns. PCA dimensions are 4
KNN: Fitting model with 310 points.
The current maximum ddg is at: 0.349790382
9 out of 319
PLS: Fitting model with 9 rows and 62 columns. PCA dimensions are 4
KNN: Fitting model with 307 points.
The current maximum ddg is at: 1.634936839
12 out of 319
PLS: Fitting model with 12 rows and 62 columns. PCA dimensions are 4
KNN: Fitting model with 304 points.
The current maximum ddg is at: 1.634936839
15 out of 319
PLS: Fitting model with 15 rows and 62 columns. PCA dimensions are 4
KNN: Fitting model with 301 points.
The current maximum ddg is at: 1.634936839
18 out of 319
PLS: Fitting model with 18 rows and 62 columns. PCA dimensions are 4
KNN: Fitting model with 298 points.
The current maximum ddg is at: 1.634936839
21 out of 319
PLS: Fitting model with 21 rows and 62 columns. PCA 

KeyboardInterrupt: 

In [123]:
# sample prediction with model
properties1 = (np.array([ 0.75813041,  0.67727654,  0.68692902,  0.57403813, -0.39901283,
       -0.83317389, -0.69366434, -0.76423736, -0.18775295, -0.89697628,
       -0.93897206, -0.64303136, -0.50508299, -0.82567658, -0.29773217,
       -0.23605932, -0.26722481,  0.02879822, -0.69189841]))
#predict_ee(pls_large_saved_model, properties1)
# random_catalyst_df = random_catalyst_df[~random_catalyst_df.index.duplicated(keep='first')]

In [124]:
# prompt: get last 3 rows of random_catalyst_df[X_COLS]
random_catalyst_df[X_COLS].tail(3)


,906,907,908,922,923,937,950,951,952,1113,...,3173,3524,3525,3547,3548,3628,3688,3916,3940,3951
Catalyst,,,,,,,,,,,,,,,,,,,,,
13_3_4,0.521739,0.521739,0.521739,0.652174,0.391304,0.695652,0.695652,0.782609,0.826087,0.521739,...,0.053217,-0.071739,-0.096,0.00,0.00,-0.071739,-0.071739,-0.004696,0.00,0.00
18_11_1,0.000000,0.000000,0.200000,0.400000,0.200000,0.200000,0.200000,0.000000,0.000000,0.200000,...,0.038000,0.000000,-0.030,0.00,0.00,-0.030000,0.000000,0.090000,0.00,0.00
11_1_4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.030000,0.030000,0.060,0.03,0.03,0.000000,0.060000,-0.148800,-0.06,0.03


In [220]:
max_cycles = 2
current_cycle = 0
catalysts_made_per_cycle = []


## outer while loop
while current_cycle < max_cycles:
  found_max_ddG = False  # Flag to control the inner loop
  random_catalyst_df = df.sample(START_SAMPLE_SIZE)

  ## inner while loop
  while(len(random_catalyst_df) < len(df)):
    print("The current maximum ddg is at: " + str(random_catalyst_df['ddG'].max()))
    print(str(len(random_catalyst_df)) + " out of " + str(len(df)))
    what_to_optimize = random_catalyst_df[X_COLS]
    if OPTIMIZE_STRATEGY == "NEWLY_ADDED":
        what_to_optimize = what_to_optimize.tail(START_SAMPLE_SIZE)
    new_optimized_catalysts = optimize_catalysts(what_to_optimize,
        pls_large_saved_model if MODEL_REFRESH_STRATEGY == "PRETRAIN" else None)
    random_catalyst_df = process_neighbors(knn, new_optimized_catalysts, random_catalyst_df)
    # recompute knn so that we dont find the same neighbors in the next loop around
    knn = recomputeKNN(df[X_COLS], random_catalyst_df)
    if random_catalyst_df['ddG'].max() >= max_ddG:
        print("Found the highest ddG {}. Exiting.".format(str(random_catalyst_df['ddG'].max())))
        print(len(random_catalyst_df))
        catalysts_made_per_cycle.append(len(random_catalyst_df))
        found_max_ddG = True  # Set flag to true to break inner loop
        break


  current_cycle += 1
  if found_max_ddG:
        continue  # Move to the next cycle if max ddG was found

  if current_cycle == max_cycles:
    print("Maximum cycles reached. Exiting.")
    break






The current maximum ddg is at: 0.593605431
3 out of 319
PLS: Fitting model with 3 rows and 62 columns. PCA dimensions are 4
KNN: Fitting model with 313 points.
The current maximum ddg is at: 0.734203206
6 out of 319
PLS: Fitting model with 6 rows and 62 columns. PCA dimensions are 4
KNN: Fitting model with 310 points.
The current maximum ddg is at: 0.734203206
9 out of 319
PLS: Fitting model with 9 rows and 62 columns. PCA dimensions are 4
KNN: Fitting model with 307 points.
The current maximum ddg is at: 1.018120793
12 out of 319
PLS: Fitting model with 12 rows and 62 columns. PCA dimensions are 4
KNN: Fitting model with 304 points.
The current maximum ddg is at: 1.046149886
15 out of 319
PLS: Fitting model with 15 rows and 62 columns. PCA dimensions are 4
KNN: Fitting model with 301 points.
The current maximum ddg is at: 1.046149886
18 out of 319
PLS: Fitting model with 18 rows and 62 columns. PCA dimensions are 4
KNN: Fitting model with 298 points.
The current maximum ddg is at: 1.5

In [223]:
catalysts_made_per_cycle
sum_catalysts_made = 0

for i in catalysts_made_per_cycle:
  sum_catalysts_made += i

average_catalysts_made = sum_catalysts_made / len(catalysts_made_per_cycle)
print(average_catalysts_made)


85.5
